In [19]:
# Import Pustaka
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

# --- Konfigurasi Data ---
# Ganti BASE_DIR dengan jalur ke folder 'dataset' Anda
BASE_DIR = 'dataset/train' 

# Kelas koin yang akan diklasifikasikan (sesuai nama folder)
CLASSES = ['50', '100', '200', '500', '1000']

In [20]:
def extract_hu_moments(image_path):
    """
    Memuat gambar, melakukan pra-pemrosesan, dan mengekstrak Momen Invarian Hu.
    """
    # 1. Muat Gambar dan Konversi ke Grayscale
    img = cv2.imread(image_path)
    if img is None:
        return None
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 2. Pra-pemrosesan dan Segmentasi
    # Gunakan Gaussian Blur untuk mengurangi noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Gunakan Thresholding Adaptif atau Otsu untuk mendapatkan gambar biner
    # Otsu's Thresholding seringkali bagus untuk koin
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    # Optional: Erosi/Dilasi untuk membersihkan bintik kecil
    kernel = np.ones((3,3),np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    # 3. Cari Kontur
    # Diasumsikan kontur terbesar adalah koin itu sendiri
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return None
    
    # Ambil kontur terbesar
    largest_contour = max(contours, key=cv2.contourArea)
    
    # 4. Hitung Momen dan Momen Invarian Hu
    moments = cv2.moments(largest_contour)
    hu_moments = cv2.HuMoments(moments).flatten()
    
    # 5. Normalisasi Fitur (Log Transform)
    # Log transform membantu menstabilkan nilai fitur, sering digunakan pada Momen Hu
    # hu_moments_normalized = -np.sign(hu_moments) * np.log10(np.abs(hu_moments))
    epsilon = 1e-12
    hu_moments_normalized = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + epsilon)

    if np.any(np.isnan(hu_moments_normalized)) or np.any(np.isinf(hu_moments_normalized)):
        return None

    return hu_moments_normalized

# --- Proses Pengumpulan Data ---
features = []
labels = []

print("Memulai pengumpulan dan ekstraksi fitur Momen Hu...")

for i, class_name in enumerate(CLASSES):
    class_path = os.path.join(BASE_DIR, class_name)
    
    if not os.path.isdir(class_path):
        print(f"Peringatan: Folder {class_path} tidak ditemukan. Melewati.")
        continue
        
    # Iterasi melalui setiap gambar di dalam folder kelas
    image_files = [f for f in os.listdir(class_path) if f.endswith(('jpg', 'jpeg', 'png'))]
    
    # Gunakan tqdm untuk menampilkan progress bar
    for filename in tqdm(image_files, desc=f'Memproses {class_name}'):
        image_path = os.path.join(class_path, filename)
        
        hu_features = extract_hu_moments(image_path)
        
        if hu_features is not None:
            features.append(hu_features)
            labels.append(class_name)

print("\nEkstraksi fitur selesai.")
print(f"Total sampel yang diekstrak: {len(features)}")

Memulai pengumpulan dan ekstraksi fitur Momen Hu...


Memproses 1000: 100%|██████████| 28/28 [00:00<00:00, 368.61it/s]


Ekstraksi fitur selesai.
Total sampel yang diekstrak: 140


In [21]:
# Konversi list ke numpy array
X = np.array(features)
y = np.array(labels)

# Bagi data menjadi Training (80%) dan Testing (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=y) # stratify memastikan pembagian kelas yang merata

print(f"Jumlah sampel pelatihan (Train): {len(X_train)}")
print(f"Jumlah sampel pengujian (Test): {len(X_test)}")

# --- Training Model SVM ---
model = SVC(kernel='rbf', C=10, gamma='scale')
model.fit(X_train, y_train)

print("Training selesai!")

Jumlah sampel pelatihan (Train): 112
Jumlah sampel pengujian (Test): 28
Training selesai!


In [22]:
# --- Prediksi dan Evaluasi ---

# Prediksi pada data pengujian
y_pred = model.predict(X_test)

print("\n--- Laporan Klasifikasi SVM ---")
print(classification_report(y_test, y_pred, target_names=CLASSES))

print("\n--- Confusion Matrix ---")
# Confusion Matrix menunjukkan seberapa banyak prediksi yang benar vs salah per kelas
cm = confusion_matrix(y_test, y_pred, labels=CLASSES)
print(cm)


--- Laporan Klasifikasi SVM ---
              precision    recall  f1-score   support

          50       0.20      0.33      0.25         6
         100       0.00      0.00      0.00         5
         200       0.67      0.67      0.67         6
         500       0.50      0.17      0.25         6
        1000       0.56      1.00      0.71         5

    accuracy                           0.43        28
   macro avg       0.38      0.43      0.38        28
weighted avg       0.39      0.43      0.38        28


--- Confusion Matrix ---
[[1 3 1 1 0]
 [0 2 1 2 1]
 [0 2 4 0 0]
 [0 0 0 5 0]
 [1 3 0 1 0]]


In [23]:
# Import joblib untuk menyimpan model
import joblib

# Simpan model yang sudah dilatih (untuk digunakan nanti)
MODEL_FILENAME = 'svm_coin_classifier.joblib'
joblib.dump(model, MODEL_FILENAME)
print(f"\nModel telah disimpan sebagai: {MODEL_FILENAME}")

# --- Simulasi Klasifikasi Gambar Baru ---

# Anda bisa mengganti ini dengan path gambar koin baru dari user
# new_coin_path = 'path/to/new/user/coin.jpg' 
# Anggap saja kita mengambil salah satu gambar dari set pengujian sebagai contoh:
# Gunakan path gambar koin 500 sebagai contoh (Anda harus mendapatkan path gambar sebenarnya)
# Contoh path:
# example_new_coin_path = os.path.join(BASE_DIR, '500', 'contoh_koin_500.jpg')

# --- Contoh Kode Klasifikasi ---
def classify_new_coin(image_path, trained_model):
    """Melakukan prediksi pada gambar koin baru."""
    
    # 1. Ekstrak fitur
    new_features = extract_hu_moments(image_path)
    
    if new_features is None:
        return "Gagal memproses gambar (kontur tidak ditemukan)."
        
    # 2. Ubah format menjadi array 2D (dibutuhkan oleh Scikit-learn)
    new_features = new_features.reshape(1, -1)
    
    # 3. Prediksi
    prediction = trained_model.predict(new_features)
    
    return prediction[0]

# Memuat model yang disimpan
# loaded_model = joblib.load(MODEL_FILENAME)

# Lakukan prediksi (gunakan model 'model' yang masih ada di memori untuk demo)
# KLASIFIKASI_HASIL = classify_new_coin(example_new_coin_path, model) 
# print(f"\nKoin baru diklasifikasikan sebagai: {KLASIFIKASI_HASIL}")


Model telah disimpan sebagai: svm_coin_classifier.joblib


In [24]:
import gradio as gr

def detect_and_label(img):
  # convert PIL → OpenCV
  img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

  circles = cv2.HoughCircles(
    gray,
    cv2.HOUGH_GRADIENT,
    dp=1,
    minDist=50,
    param1=100,
    param2=30,
    minRadius=20,
    maxRadius=200
  )

  output = img.copy()

  if circles is not None:
    circles = np.uint16(np.around(circles))
    for (x, y, r) in circles[0, :]:
      roi = gray[y-r:y+r, x-r:x+r]
      if roi.size == 0:
        continue

      roi_resized = cv2.resize(roi, (128,128)).flatten().reshape(1, -1)
      pred = model.predict(roi_resized)[0]

      cv2.circle(output, (x, y), r, (0, 0, 255), 3)
      cv2.putText(output, str(pred), (x - 20, y - r - 10),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

  output = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
  return output


gr.Interface(
  fn=detect_and_label,
  inputs=gr.Image(type="pil", label="Upload Foto Koin"),
  outputs=gr.Image(type="numpy", label="Hasil Deteksi"),
  title="Klasifikasi Koin Rupiah",
  description="Upload gambar koin, sistem akan mendeteksi dan memberi label nominal."
).launch()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\queueing.py", line 763, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\blocks.py", line 2125, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python313\Lib\site-packages\gradio\blocks.py", line 1607, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
  